In [1]:
from sklearn import neighbors
from sklearn import preprocessing
from arch.bootstrap import StationaryBootstrap
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import math
import sys
import random
from itertools import chain

In [2]:
#read training and cross validation set
total = pd.read_csv("data/time_space.csv")
n = total.shape[0]
num_tract = 1758

total = total.drop(total.columns[0], axis = 1) #delete index

tract_id = total['GEOID']
total = total.drop(total.columns[0], axis = 1) #delete tract_geoid
    
mp_date = total['MP_DATE']
origin = dt.datetime.strptime(total.loc[0, 'MP_DATE'],'%Y-%m-%d').date()
for i in range(n):
    now = dt.datetime.strptime(total.loc[i, 'MP_DATE'],'%Y-%m-%d').date()
    total.loc[i, 'MP_DATE'] = (now - origin).days/30.0    
values = total['values']
total = total.drop(total.columns[3], axis = 1) #delete values

test = pd.read_csv("data/test_data.csv")
m = test.shape[0]
test = test.drop(test.columns[0], axis = 1) #delete index
test_geoid = test['GEOID']
test = test.drop(test.columns[0], axis = 1) #delete GEOID
for i in range(m):
    now = dt.datetime.strptime(test.loc[i, 'MP_DATE'],'%Y-%m-%d').date()
    test.loc[i, 'MP_DATE'] = (now - origin).days/30.0

In [3]:
#another way of scale
total['MP_DATE'] = preprocessing.scale(total['MP_DATE'])
total['lon'] = preprocessing.scale(total['lon'])
total['lat'] = preprocessing.scale(total['lat'])
test['MP_DATE'] = preprocessing.scale(test['MP_DATE'])
test['lon'] = preprocessing.scale(test['lon'])
test['lat'] = preprocessing.scale(test['lat'])

#set the number of ensemble samples
s = 50

C:\Users\green\Anaconda3\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, _DataConversionWarning)


In [4]:
#scale latitudes and lontitudes
original_mpdate = total['MP_DATE']
original_lat = total['lat']
original_lon = total['lon']
original_lat_test = test['lat']
original_lon_test = test['lon']
weight = 1
total['lat'] = weight * total['lat']
total['lon'] = weight * total['lon']
test['lat'] = weight * test['lat']
test['lon'] = weight * test['lon']

#set the number of ensemble samples
s = 50 

In [5]:
blocksize = 12
#get block bootstrapping samples
for j in range(num_tract):
    sub_data = total.iloc[63*j:63*(j+1)]
    bs = StationaryBootstrap(blocksize, sub_data)
    if j == 0:
        train_lst = [data[0][0] for data in bs.bootstrap(s)]
    else:
        i = 0
        for data in bs.bootstrap(s):
            train_lst[i] = train_lst[i].append(data[0][0])
            i += 1
print("Finish Bootstrapping")
            
#get unselected samples as my cross validation
ind_all = set(total.index)
ind_diff = []
possible_oob = set(chain.from_iterable([list(range(63*k+24,63*(k+1)+1)) for k in range(1758)]))
d_oob_learner = dict()
d_oob_cv = dict()

for i in range(s):
    ind_unselected = list(possible_oob.intersection(ind_all - set(train_lst[i].index)))
    print(len(ind_unselected))
    for item in ind_unselected:
        try:
            d_oob_learner[item] += [i]
        except:
            d_oob_learner[item] = [i]
    d_oob_cv[i] = ind_unselected
print("Finish collecting feasible OOB for CV and prediction")

Finish Bootstrapping
20909
21099
21442
20508
21131
21081
20642
20351
21072
21164
20828
20502
21204
21276
20815
21126
20741
20445
20892
21225
20789
20408
20846
20808
20896
20593
20337
21021
20672
20635
20728
20602
20786
20788
20881
20728
21124
21176
21186
21071
20766
21072
20806
20817
20865
20346
20865
20933
20880
20784
Finish collecting feasible OOB for CV and prediction


In [6]:
#find best k for each bootstrapping sample --> model parameter
best_k_lst = []
best_mae_lst = []
symbol_lst = []
y_ = []
for i in range(s):
    train_y = train_lst[i]['dv'] #get dependent variable
    train_x = train_lst[i].drop(train_lst[i].columns[3], axis = 1) #delete dependent variables

    best_mae = sys.maxsize
    for k in range(1, 11, 1):
        knn = neighbors.KNeighborsRegressor(k, weights ='distance')
        #y_ = []
        y_mean = 0
        y_symbol = 0
        cv_times = range(50,62)
        for cv_time in cv_times:
            train_id = list(chain.from_iterable([list(range(63*k,(63*k+cv_time))) for k in range(num_tract)]))
            test_id = [63*k+cv_time for k in range(num_tract)]
            #print(test_id)
            y_predict = knn.fit(train_x.iloc[train_id], train_y.iloc[train_id]).predict(train_x.iloc[test_id])
            y_ += [y_predict]
            mae = np.mean(abs(y_predict - train_y.iloc[test_id]))
            y_mean += mae
            y_symbol += sum(y_predict*train_y.iloc[test_id]>=0)/len(y_predict)
        y_mean = y_mean / len(cv_times)
        y_symbol = y_symbol / len(cv_times)
        if y_mean < best_mae:
            best_mae = y_mean
            best_k = k
            symbol = y_symbol
            print((i, k, best_mae))
    best_mae_lst.append(best_mae)
    best_k_lst.append(best_k)
    symbol_lst.append(symbol)

(0, 1, 2.0240339443430178)
(0, 3, 1.9736216082734153)
(0, 4, 1.9467304512848067)
(0, 5, 1.9411084857573628)
(1, 1, 3.408618911570294)
(1, 2, 3.2274723593151244)
(2, 1, 1.5599494022378713)
(2, 2, 1.5230868512059674)
(3, 1, 1.5324756791551104)
(4, 1, 3.3448452023419724)
(4, 2, 3.065067503826059)
(4, 4, 3.03639925542222)
(4, 5, 2.95730741323411)
(4, 8, 2.9409465178889485)
(5, 1, 2.503162457926136)
(5, 2, 2.4285599918317966)
(5, 3, 2.150208533362352)
(5, 4, 2.035142041380085)
(5, 5, 1.95296923063339)
(6, 1, 1.842098430275523)
(6, 2, 1.7577865574543943)
(6, 3, 1.7067081109199245)
(7, 1, 1.8112564472711685)
(7, 2, 1.6986254495386988)
(7, 3, 1.6911611685294057)
(8, 1, 2.000363827301325)
(8, 2, 1.897471598409932)
(8, 6, 1.8841472017758176)
(9, 1, 1.791946273449611)
(9, 2, 1.7292841252709137)
(9, 3, 1.7209887590440063)
(9, 7, 1.7163599845782926)
(10, 1, 2.000688338781869)
(10, 2, 1.9388017318687256)
(11, 1, 2.601861230864272)
(11, 2, 2.3966897180128086)
(12, 1, 2.0448385565140765)
(12, 2, 1.956

In [7]:
lst = symbol_lst
lst

[0.95667425104285175,
 0.96264694728858557,
 0.96449563898369373,
 0.96321577550246484,
 0.95193401592718996,
 0.9569112627986347,
 0.95975540386803182,
 0.96027682973075468,
 0.95297686765263556,
 0.95563139931740615,
 0.9616514979142966,
 0.95833333333333337,
 0.95724307925673113,
 0.95648464163822522,
 0.96165149791429672,
 0.96411642017444066,
 0.95672165339400816,
 0.95477815699658697,
 0.96278915434205548,
 0.95401971937808128,
 0.94994311717861202,
 0.95458854759196077,
 0.95951839211224865,
 0.95994501327265835,
 0.96321577550246484,
 0.95335608646188852,
 0.96027682973075479,
 0.96245733788395882,
 0.95947098976109191,
 0.96169890026545313,
 0.95913917330299581,
 0.95790671217292378,
 0.95951839211224887,
 0.96127227910504365,
 0.96027682973075457,
 0.95217102768297313,
 0.95842813803564653,
 0.95183921122487691,
 0.95913917330299592,
 0.96392681076981424,
 0.96146188850967018,
 0.9643060295790673,
 0.96013462267728478,
 0.96169890026545313,
 0.95766970041714072,
 0.9631209708

In [8]:
#draw MAE
plt.figure(figsize=(8,6), dpi=60)
plt.plot(range(s), best_mae_lst, 'k')
plt.xlabel('Index of bootstrapping samles')
plt.ylabel('MAE')
plt.savefig("final_results/MAE.png", dpi = 60)
np.mean(best_mae_lst)

2.1337667001739997

In [9]:
#get predictions
y_pre_lst = []
for i in range(s):
    train_y = train_lst[i]['dv'] #get dependent variable
    train_x = train_lst[i].drop(train_lst[i].columns[3], axis = 1) #delete dependent variables
    knn = neighbors.KNeighborsRegressor(best_k_lst[i], weights ='distance')
    y_pre = knn.fit(train_x, train_y).predict(test)
    y_pre_lst.append(y_pre)
y_pre_mat = np.array(y_pre_lst)
y_final_pre = np.mean(y_pre_mat, 0)

#get prediction in 2015.4
y_final_pre_2 = [y_final_pre[12*i] for i in range(1758)]

In [52]:
pd.Series(y_final_pre, index = range(1, 21097, 1)).to_csv("final_results/all_predictions.csv")

In [11]:
# draw graphs to show predictions of each tract
for i in range(1758):
    filename = "final_results/figure/%d.png" % i
    plt.figure(figsize=(8,6), dpi=60)
    plt.plot(range(12), y_final_pre[i*12:(i+1)*12], 'k')
    plt.xlabel('Date')
    plt.ylabel('Future Annual Return')
    plt.savefig(filename, dpi = 60)

C:\Users\green\Anaconda3\lib\site-packages\matplotlib\pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [15]:
#output average increase rates for each tract
unique_tract = list((set(tract_id)))
tract_rate = []
tract_2015_4 = []
for i in range(1758):
    future_rate = y_final_pre[i*12:(i+1)*12]
    tract_rate.append(np.mean(future_rate))
    tract_2015_4.append(y_final_pre_2[i])
data = {'Future Annual Increase Rate': tract_rate,
        'TRACT_GEOID': unique_tract}
data1 = {'Future Annual Increase Rate': tract_2015_4,
        'TRACT_GEOID': unique_tract}
df = pd.DataFrame(data, index = range(1, 1759, 1))
df_2016 = pd.DataFrame(data1, index = range(1, 1759, 1))
df.to_csv('final_results/final_prediction.csv')
df_2016.to_csv('final_results/2015_4_prediction.csv')

In [19]:
# final recommendations
recommend = dict()
recommend['invest'] = []
recommend['hold'] = []
recommend['divest'] = []
unique_tract = list((set(tract_id)))
tract_rate = []
for i in range(1758):
    future_rate = y_final_pre[i*12:(i+1)*12]
    tract_rate.append(np.mean(future_rate))
    if future_rate[1] < future_rate[0]:
        recommend['divest'].append(unique_tract[i])
    elif future_rate[0] > 0.2:
        recommend['invest'].append(unique_tract[i])
    else:
        recommend['hold'].append(unique_tract[i])

In [27]:
#output invest decisions for each tract
n_1 = len(recommend['divest'])
n_2 = len(recommend['hold'])
n_3 = len(recommend['invest'])
n_max = max(n_1, n_2, n_3)
for i in range(n_max - n_1):
    recommend['divest'].append(0)
for i in range(n_max - n_3):
    recommend['invest'].append(0)
df_2 = pd.DataFrame(recommend, index = range(1, n_max+1, 1))
df_2.to_csv('invest_decisions.csv')

In [26]:
n_1

840

In [28]:
new_df = dict()
for tract in unique_tract:
    if tract in recommend['invest']:
        new_df[tract] = 3
    elif tract in recommend['hold']:
        new_df[tract] = 2
    else:
        new_df[tract] = 1
final_df = dict()
final_df['GEOID'] = [k for k in new_df.keys()]
final_df['decision'] = [v for v in new_df.values()]

In [29]:
df_3 = pd.DataFrame(final_df, index = range(1, 1759, 1))
df_3.to_csv('final_results/tract_decisions.csv')

In [5]:
#knn without ensemble
train = total.loc[total['MP_DATE'] < 1.098845, :]
cv = total.loc[total['MP_DATE'] >= 1.098845, :]
train_y = train['dv']
train_x = train.drop(train.columns[3], axis = 1)
cv_y = cv['dv']
cv_x = cv.drop(cv.columns[0], axis = 1)

best_rss_no = sys.maxsize
best_k_no = 1

for i in range(1, 16, 1):
    knn = neighbors.KNeighborsRegressor(i, weights ='distance')
    y_ = knn.fit(train_x, train_y).predict(cv_x)
    mid = np.mean(abs(y_ - cv_y))
    if mid < best_rss_no:
        best_rss_no = mid
        best_k_no = i
    print((i, best_rss_no))

(1, 6.4968118110876905)
(2, 6.301173121467906)
(3, 6.173240622644924)
(4, 5.966675882056235)
(5, 5.858295697876859)
(6, 5.74428054377989)
(7, 5.638902350126948)
(8, 5.599278349793313)
(9, 5.557570289779132)
(10, 5.517544386503674)
(11, 5.517544386503674)
(12, 5.517544386503674)
(13, 5.499337073192289)
(14, 5.499337073192289)
(15, 5.499337073192289)


In [6]:
#get model performance without ensemble
knn = neighbors.KNeighborsRegressor(best_k_no, weights ='distance')
y_no = knn.fit(train_x, train_y).predict(cv_x)
accuracy = (sum(y_no*cv_y>0)/len(cv_y))
MAE_no = np.mean(abs(y_no - cv_y))

In [25]:
#draw correct rate (comparison)
plt.figure(figsize=(8,6), dpi=60)
plt.plot(range(s), lst, 'k')
plt.plot(range(s), symbol_lst*s, 'k', color = 'r')
plt.xlabel('Index of bootstrapping samples')
plt.ylabel('Accuracy')
plt.savefig("final_results/accuracy2.png", dpi = 60)
np.mean(symbol_lst)

0.94539249146757676

In [63]:
#draw correct rate (no comparison)
plt.figure(figsize=(8,6), dpi=60)
plt.plot(range(s), symbol_lst, 'k')
plt.xlabel('Index of bootstrapping samles')
plt.ylabel('Accuracy')
plt.savefig("final_results/accuracy_scale.png", dpi = 60)
np.mean(symbol_lst)

C:\Users\green\Anaconda3\lib\site-packages\matplotlib\pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


0.95995544178991266

In [252]:
data_15_3 = pd.DataFrame()
for j in range(num_tract):
    data_15_3 = data_15_3.append(total.iloc[62+63*j])
train_15_3 = data_15_3.drop(data_15_3.columns[3], axis = 1)

In [254]:
#get predictions
y_pre_lst = []
for i in range(s):
    train_y = train_lst[i]['dv'] #get dependent variable
    train_x = train_lst[i].drop(train_lst[i].columns[3], axis = 1) #delete dependent variables
    knn = neighbors.KNeighborsRegressor(best_k_lst[i], weights ='distance')
    y_pre = knn.fit(train_x, train_y).predict(train_15_3)
    y_pre_lst.append(y_pre)
y_pre_mat = np.array(y_pre_lst)
y_final_pre = np.mean(y_pre_mat, 0)

In [255]:
#output average increase rates for each tract
unique_tract = list((set(tract_id)))
tract_2015_3 = []
for i in range(1758):
    tract_rate.append(np.mean(future_rate))
    tract_2015_3.append(y_final_pre_2[i])
data1 = {'Future Annual Increase Rate': tract_2015_3,
        'TRACT_GEOID': unique_tract}
df_2015_3 = pd.DataFrame(data1, index = range(1, 1759, 1))
df_2015_3.to_csv('final_results/2015_3_prediction.csv')

In [59]:
total.loc[51]

MP_DATE    1.098845
lon        0.344138
lat        1.021606
dv        -0.398340
Name: 51, dtype: float64

In [ ]:
for blocksize in range(25):
    print("Current Block Size: "+str(blocksize))
    
    #get 10 block bootstrapping samples
    for j in range(num_tract):
        sub_data = total.iloc[63*j:63*(j+1)]
        bs = StationaryBootstrap(blocksize, sub_data)
        if j == 0:
            train_lst = [data[0][0] for data in bs.bootstrap(s)]
        else:
            i = 0
            for data in bs.bootstrap(s):
                train_lst[i] = train_lst[i].append(data[0][0])
                i += 1
    print("Finish Bootstrapping")

            
    #get unselected samples as my cross validation
    ind_all = set(total.index)
    ind_diff = []
    possible_oob = set(chain.from_iterable([list(range(63*k+24,63*(k+1)+1)) for k in range(1758)]))
    d_oob_learner = dict()
    d_oob_cv = dict()

    for i in range(s):
        ind_unselected = list(possible_oob.intersection(ind_all - set(train_lst[i].index)))
        print(len(ind_unselected))
        for item in ind_unselected:
            try:
                d_oob_learner[item] += [i]
            except:
                d_oob_learner[item] = [i]
        d_oob_cv[i] = ind_unselected
    print("Finish collecting feasible OOB for CV and prediction")
    
    
    #find best k for each bootstrapping sample --> model parameter
    best_k_lst = []
    best_mae_lst = []
    symbol_lst = []
    y_ = []
    for i in range(s):
        train_y = train_lst[i]['dv'] #get dependent variable
        train_x = train_lst[i].drop(train_lst[i].columns[3], axis = 1) #delete dependent variables

        best_mae = sys.maxsize
        for k in range(1, 32, 5):
            knn = neighbors.KNeighborsRegressor(k, weights ='distance')
            y_ = []
            y_mean = 0
            y_symbol = 0
            cv_times = range(50,62)
            for cv_time in cv_times:
                train_id = list(chain.from_iterable([list(range(63*k,(63*k+cv_time))) for k in range(num_tract)]))
                test_id = [63*k+cv_time for k in range(num_tract)]
            #print(test_id)
                y_predict = knn.fit(train_x.iloc[train_id], train_y.iloc[train_id]).predict(train_x.iloc[test_id])
                y_ += [y_predict]
                mae = np.mean(abs(y_predict - train_y.iloc[test_id]))
                y_mean += mae
                y_symbol += sum(y_predict*train_y.iloc[test_id]>=0)/len(y_predict)
            y_mean = y_mean / len(cv_times)
            y_symbol = y_symbol / len(cv_times)
            if y_mean < best_mae:
                best_mae = y_mean
                best_k = k
                symbol = y_symbol
                print((i, k, best_mae))
        best_mae_lst.append(best_mae)
        best_k_lst.append(best_k)
        symbol_lst.append(symbol)



In [23]:
#find best k for sample without bagging
best_k_lst = []
best_mae_lst = []
symbol_lst = []
y_ = []
for i in range(1):
    train_y = train_lst[i]['dv'] #get dependent variable
    train_x = train_lst[i].drop(train_lst[i].columns[3], axis = 1) #delete dependent variables

    best_mae = sys.maxsize
    for k in range(1, 11, 1):
        knn = neighbors.KNeighborsRegressor(k, weights ='distance')
        #y_ = []
        y_mean = 0
        y_symbol = 0
        cv_times = range(50,62)
        for cv_time in cv_times:
            train_id = list(chain.from_iterable([list(range(63*k,(63*k+cv_time))) for k in range(num_tract)]))
            test_id = [63*k+cv_time for k in range(num_tract)]
            #print(test_id)
            y_predict = knn.fit(train_x.iloc[train_id], train_y.iloc[train_id]).predict(train_x.iloc[test_id])
            y_ += [y_predict]
            mae = np.mean(abs(y_predict - train_y.iloc[test_id]))
            y_mean += mae
            y_symbol += sum(y_predict*train_y.iloc[test_id]>=0)/len(y_predict)
        y_mean = y_mean / len(cv_times)
        y_symbol = y_symbol / len(cv_times)
        if y_mean < best_mae:
            best_mae = y_mean
            best_k = k
            symbol = y_symbol
            print((i, k, best_mae))
    best_mae_lst.append(best_mae)
    best_k_lst.append(best_k)
    symbol_lst.append(symbol)

(0, 1, 4.447023224609177)
(0, 2, 3.9923118347370257)
(0, 3, 3.780615891468511)
(0, 5, 3.739009739413857)
(0, 6, 3.6847373402018495)
(0, 7, 3.628582636651039)
(0, 8, 3.608511551253096)
(0, 9, 3.583130223809627)
(0, 10, 3.5754736168227)


In [24]:
symbol_lst

[0.94539249146757676]